In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
list_files = [os.path.join("../data/raw", i) for i in os.listdir("../data/raw")]
size_files = [os.stat(i).st_size/(1024*1024) for i in list_files]
print(dict(zip(list_files, size_files)))

{'../data/raw/train_labels.csv': 29.322805404663086, '../data/raw/test_data.csv': 32257.8899641037, '../data/raw/sample_submission.csv': 59.07977104187012, '../data/raw/train_data.csv': 15633.853614807129}


In [4]:
train_label_df = pd.read_csv("../data/raw/train_labels.csv")

In [5]:
train_label_df.head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [11]:
train_label_df.shape

(458913, 2)

In [12]:
train_label_df['target'].value_counts(dropna=False, normalize=True)

target
0    0.741066
1    0.258934
Name: proportion, dtype: float64

In [7]:
df_1lacs = pd.read_csv("../data/raw/train_data.csv", nrows=200000)

In [8]:
df_1lacs.shape

(200000, 190)

In [9]:
df_1lacs.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

D_* = Delinquency variables
S_* = Spend variables
P_* = Payment variables
B_* = Balance variables
R_* = Risk variables
with the following features being categorical:

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
Your task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.

In [10]:
cols = df_1lacs.columns

In [12]:
delinquency_cols = [i for i in cols if i.startswith('D')]
spend_cols = [i for i in cols if i.startswith('S')]
payment_cols = [i for i in cols if i.startswith('P')]
balance_cols = [i for i in cols if i.startswith('B')]
risk_cols = [i for i in cols if i.startswith('R')]

In [13]:
len(delinquency_cols), len(spend_cols), len(payment_cols), len(balance_cols), len(risk_cols)

(96, 22, 3, 40, 28)

Delinquency variables: The term delinquent refers to the state of being in arrears. When someone is delinquent, they are past due on their financial obligation(s), such as a loan, credit card, or bond payments. This means a borrower's payments are not made to satisfy their debt(s) in a timely manner. Delinquent entities can be either individuals or corporations. Financial delinquency often leads to default if the arrears aren't brought up to date.

In [17]:
df_1lacs[delinquency_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 96 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_39    200000 non-null  float64
 1   D_41    199928 non-null  float64
 2   D_42    29193 non-null   float64
 3   D_43    139942 non-null  float64
 4   D_44    190071 non-null  float64
 5   D_45    199928 non-null  float64
 6   D_46    156105 non-null  float64
 7   D_47    200000 non-null  float64
 8   D_48    173989 non-null  float64
 9   D_49    20462 non-null   float64
 10  D_50    86464 non-null   float64
 11  D_51    200000 non-null  float64
 12  D_52    198991 non-null  float64
 13  D_53    52597 non-null   float64
 14  D_54    199928 non-null  float64
 15  D_55    193476 non-null  float64
 16  D_56    91119 non-null   float64
 17  D_58    200000 non-null  float64
 18  D_59    196233 non-null  float64
 19  D_60    200000 non-null  float64
 20  D_61    178557 non-null  float64
 21  D_62    17

In [26]:
# Column Profiling: data type, null percentage, describe (min, max, [5,25,50,75,95], mean, std, variance)

def col_profiling(col_series):
    res_dict = {'datatype': col_series.dtype,
                'null_pct': col_series.isna().sum()*100/len(col_series),
                'min_val': col_series.min(),
                'max_val': col_series.max(),
                'mean_val': col_series.mean(),
                'std': col_series.std(),
                'variance': col_series.std(),
                '5_pct': col_series.quantile(q=0.05),
                '25_pct': col_series.quantile(q=0.25),
                '50_pct': col_series.quantile(q=0.5),
                '75_pct': col_series.quantile(q=0.75),
                '95_pct': col_series.quantile(q=0.95),
                'num_uniq': col_series.nunique()}
    return res_dict

In [27]:
# Column Profiling on delinquency variables
ls_delinquency_variable = []
for item in delinquency_cols:
    if df_1lacs[item].dtype == np.float64:
        res = col_profiling(df_1lacs[item])
        res['column_name'] = item
        ls_delinquency_variable.append(res)
    else:
        print(f"categorical column is {item}")

categorical column is D_63
categorical column is D_64


In [28]:
col_meta = pd.DataFrame(ls_delinquency_variable).set_index('column_name')

In [29]:
col_meta

,datatype,null_pct,min_val,max_val,mean_val,std,variance,5_pct,25_pct,50_pct,75_pct,95_pct,num_uniq
column_name,,,,,,,,,,,,,
D_39,float64,0.0000,3.892609e-07,5.331360,0.153413,0.272381,0.272381,0.000905,0.004541,0.009066,0.236017,0.654666,200000
D_41,float64,0.0360,5.627163e-08,6.798167,0.062218,0.208162,0.208162,0.000586,0.002898,0.005769,0.008653,0.401103,199928
D_42,float64,85.4035,-2.188914e-04,3.252056,0.181170,0.216851,0.216851,0.007143,0.039449,0.120749,0.250728,0.544487,29193
D_43,float64,30.0290,8.705647e-07,9.089694,0.156035,0.216974,0.216974,0.009607,0.042353,0.088289,0.185098,0.522488,139942
D_44,float64,4.9645,1.031370e-08,2.880841,0.121163,0.222514,0.222514,0.000784,0.003895,0.007755,0.132351,0.626592,190071
...,...,...,...,...,...,...,...,...,...,...,...,...,...
D_141,float64,1.7515,5.642931e-08,1.174753,0.165382,0.349056,0.349056,0.000621,0.003033,0.006063,0.009094,0.952388,196497
D_142,float64,82.8220,-1.444070e-02,1.751388,0.392596,0.238923,0.238923,0.040042,0.196174,0.382038,0.566102,0.772431,34356
D_143,float64,1.7515,1.653580e-08,1.010000,0.179736,0.379746,0.379746,0.000610,0.003034,0.006078,0.009089,1.007163,196497


What are some columns, I will not consider for analysis:

1. All unique: In my opinion these are kind of unique Ids.
2. All same: These are constants
3. Pseudo constants: These are kind of constants, if we are getting more than 95% same value.
4. All null: Empty columns
5. cols with most of the values are nulls (we can take 95% as cut off)

In [30]:
# Column Profiling on spend_cols variables
ls_spend_variable = []
for item in spend_cols:
    if df_1lacs[item].dtype == np.float64:
        res = col_profiling(df_1lacs[item])
        res['column_name'] = item
        ls_spend_variable.append(res)
    else:
        print(f"categorical column is {item}")

categorical column is S_2


In [32]:
col_meta = pd.DataFrame(ls_spend_variable).set_index('column_name')
col_meta

,datatype,null_pct,min_val,max_val,mean_val,std,variance,5_pct,25_pct,50_pct,75_pct,95_pct,num_uniq
column_name,,,,,,,,,,,,,
S_3,float64,18.8325,-2.547070e-01,2.918675,0.227386,0.197101,0.197101,0.060660,0.127432,0.164248,0.260202,0.607962,162335
S_5,float64,0.0000,1.465230e-07,143.342047,0.088260,0.647703,0.647703,0.001208,0.005646,0.013417,0.069816,0.354358,200000
S_6,float64,0.0000,1.522788e-07,1.010000,0.248121,0.428985,0.428985,0.000655,0.003303,0.006617,0.009910,1.007958,200000
S_7,float64,18.8325,-1.807581e-01,2.611836,0.224602,0.208025,0.208025,0.046565,0.093870,0.141805,0.298479,0.633364,162335
S_8,float64,0.0000,1.342140e-07,1.231351,0.325144,0.302428,0.302428,0.001510,0.007548,0.321219,0.494391,0.859489,200000
S_9,float64,53.2990,3.154521e-06,2.839094,0.076049,0.200230,0.200230,0.003808,0.009853,0.020087,0.053968,0.317210,93402
S_11,float64,0.0000,-1.997693e-01,3.801432,0.376537,0.191383,0.191383,0.127043,0.282564,0.289744,0.449325,0.729158,200000
S_12,float64,0.0000,-3.922996e-01,16.746990,0.239863,0.205906,0.205906,0.152419,0.186901,0.190822,0.207764,0.508066,200000
S_13,float64,0.0000,5.237888e-08,1.009999,0.254424,0.297776,0.297776,0.000964,0.004829,0.009755,0.427756,0.850475,200000


In [33]:
# Column Profiling on spend_cols variables
ls_balance_variable = []
for item in balance_cols:
    if df_1lacs[item].dtype == np.float64:
        res = col_profiling(df_1lacs[item])
        res['column_name'] = item
        ls_balance_variable.append(res)
    else:
        print(f"categorical column is {item}")

categorical column is B_31


In [34]:
col_meta = pd.DataFrame(ls_balance_variable).set_index('column_name')
col_meta

,datatype,null_pct,min_val,max_val,mean_val,std,variance,5_pct,25_pct,50_pct,75_pct,95_pct,num_uniq
column_name,,,,,,,,,,,,,
B_1,float64,0.0000,-8.993963e-01,1.324053,0.125356,0.213396,0.213396,0.002134,0.008858,0.031935,0.129569,0.610299,200000
B_2,float64,0.0360,3.308280e-06,1.010000,0.616971,0.402275,0.402275,0.019186,0.100550,0.814164,1.002240,1.008441,199928
B_3,float64,0.0360,1.042180e-07,1.625262,0.134392,0.234849,0.234849,0.001220,0.005291,0.009873,0.163591,0.703738,199928
B_4,float64,0.0000,4.744112e-07,2.789190,0.174172,0.223372,0.223372,0.003606,0.027683,0.083329,0.240015,0.631093,200000
B_5,float64,0.0000,1.730131e-07,80.996800,0.081480,0.502791,0.502791,0.001915,0.007256,0.015140,0.052804,0.334800,200000
B_6,float64,0.0035,-4.479462e-03,79.091481,0.146276,0.537191,0.537191,0.006215,0.020036,0.081913,0.191341,0.330372,199993
B_7,float64,0.0000,-8.528932e-01,1.252750,0.188786,0.232391,0.232391,0.008839,0.028490,0.076779,0.277696,0.692244,200000
B_8,float64,0.4240,6.047988e-08,1.010775,0.452783,0.497188,0.497188,0.000895,0.004526,0.009054,1.004216,1.008680,199152
B_9,float64,0.0000,6.195194e-08,17.515247,0.190689,0.281801,0.281801,0.001151,0.005771,0.026296,0.340702,0.651531,200000


In [35]:
# Column Profiling on payment variables
ls_payment_variable = []
for item in payment_cols:
    if df_1lacs[item].dtype == np.float64:
        res = col_profiling(df_1lacs[item])
        res['column_name'] = item
        ls_payment_variable.append(res)
    else:
        print(f"categorical column is {item}")

In [37]:
col_meta = pd.DataFrame(ls_payment_variable).set_index('column_name')
col_meta

,datatype,null_pct,min_val,max_val,mean_val,std,variance,5_pct,25_pct,50_pct,75_pct,95_pct,num_uniq
column_name,,,,,,,,,,,,,
P_2,float64,0.7710,-3.830190e-01,1.009998,0.653527,0.246135,0.246135,0.216871,0.476334,0.691541,0.863455,0.975531,198458
P_3,float64,5.4435,-1.055714e+00,2.027742,0.600592,0.171354,0.171354,0.302522,0.539831,0.618742,0.684020,0.830895,189113
P_4,float64,0.0000,6.667930e-08,1.198705,0.146603,0.340980,0.340980,0.000584,0.002925,0.005853,0.008792,0.964440,200000


In [38]:
# Column Profiling on risk variables
ls_risk_variable = []
for item in risk_cols:
    if df_1lacs[item].dtype == np.float64:
        res = col_profiling(df_1lacs[item])
        res['column_name'] = item
        ls_risk_variable.append(res)
    else:
        print(f"categorical column is {item}")

In [39]:
col_meta = pd.DataFrame(ls_risk_variable).set_index('column_name')
col_meta

,datatype,null_pct,min_val,max_val,mean_val,std,variance,5_pct,25_pct,50_pct,75_pct,95_pct,num_uniq
column_name,,,,,,,,,,,,,
R_1,float64,0.0000,2.962930e-08,2.507711,0.081215,0.230995,0.230995,0.000579,0.002900,0.005793,0.008685,0.507647,200000
R_2,float64,0.0000,8.265748e-10,1.009999,0.049152,0.205465,0.205465,0.000517,0.002610,0.005230,0.007842,0.009938,200000
R_3,float64,0.0000,2.238322e-09,6.709854,0.121991,0.214724,0.214724,0.000945,0.004699,0.009367,0.200358,0.504391,200000
R_4,float64,0.0000,3.459658e-09,1.009994,0.032688,0.164094,0.164094,0.000511,0.002577,0.005140,0.007719,0.009775,200000
R_5,float64,0.0000,7.782544e-08,13.005083,0.036992,0.262912,0.262912,0.000515,0.002568,0.005154,0.007729,0.009794,200000
R_6,float64,0.0000,1.540863e-07,14.448172,0.058706,0.625465,0.625465,0.000521,0.002602,0.005208,0.007798,0.009870,200000
R_7,float64,0.0000,1.198959e-08,252.765475,0.093112,1.760810,1.760810,0.000514,0.002573,0.005144,0.007719,0.009773,200000
R_8,float64,0.0000,6.524608e-08,15.009584,0.040451,0.311915,0.311915,0.000514,0.002563,0.005109,0.007667,0.009711,200000
R_9,float64,94.0525,3.121485e-07,1.509625,0.226718,0.183068,0.183068,0.006477,0.169109,0.172702,0.176203,0.508318,11895


Some assumptions are there, that we will validate over the full dataset. Learning till now:

1. Some unique values are there
2. Some outliers are there
3. Some cols with most of the missing values.
4. This dataset has been created by merging 5 tables: delinquency, spend, payment, balance and risk. So first task is seggrigating and joining the target with that table, just to get an overall idea of analytical values on a single group of data.
